# XAUUSD Candlestick Chart Generator
این نوت‌بوک به صورت خودکار از داده‌های شما چارت‌های کندلی تولید کرده و با توجه به آینده، برچسب خرید/فروش/نگه‌داری اختصاص می‌دهد.

In [ ]:
!pip install mplfinance pandas numpy matplotlib

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mplfinance as mpf

df = pd.read_csv(list(uploaded.keys())[0])
df.columns = ["symbol", "time", "open", "high", "low", "close", "volume"]
df["time"] = pd.to_datetime(df["time"], format="%Y%m%d%H%M%S")
df.set_index("time", inplace=True)
df = df[["open", "high", "low", "close", "volume"]].copy()
df.head()

In [ ]:
window = 60
future_shift = 10
threshold = df["close"].rolling(14).std().mean()
output_dir = "xau_charts"
os.makedirs(output_dir, exist_ok=True)

labels = []
for i in range(window, window + 20):  # فقط 20 نمونه
    segment = df.iloc[i - window:i]
    current_price = df["close"].iloc[i]
    future_price = df["close"].iloc[i + future_shift]
    delta = future_price - current_price

    if delta > threshold:
        label = "buy"
    elif delta < -threshold:
        label = "sell"
    else:
        label = "hold"

    filename = f"{label}_{i}.png"
    filepath = os.path.join(output_dir, filename)
    mpf.plot(segment, type='candle', style='charles', savefig=filepath, volume=False)
    labels.append((filename, label))

# ذخیره لیبل‌ها
label_df = pd.DataFrame(labels, columns=["filename", "label"])
label_df.to_csv(os.path.join(output_dir, "labels.csv"), index=False)
print("تصاویر و لیبل‌ها ساخته شدند.")